En este documento estamos tomando un dataset que se nos dio por medio de canvas con la informacion de diferentes cliente de una aseguradora, en la que se tiene informacion personal de los clientes asi como el costo del seguro, con esto el objetivo sera crear un modelo que nos permita predecir el costo del seguro de un cliente nuevo.

En base a lo que hemos aprendido decidi que para esta entrega realizare una red neuronal, simplemente por el hecho de que es un modelo que no hemos tenido la oportunidad de practir como otros modelos de machine learning, por lo que me parecio una buena oportunidad para aprender a usarlo.

In [69]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Importamos el dataset

In [70]:
df = pd.read_csv(r'E:\Github\Portafolio_Implementacion\ConFrameworks\insurance.csv', index_col=0)

checamos las columnas del dataset, podemos ver que no hay valores nulos y que hay 3 varaibles categoricas que necesitaremos cambiar para poder crear nuestros modelos.

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 83.6+ KB


Cambiamos las variables categoricas a numericas

In [72]:
df['sex'] = df['sex'].map({'female': 0, 'male': 1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes': 1})

df = pd.get_dummies(df, columns=['region'], drop_first=False)

creamos los conjuntos de input y output

In [73]:
X = df.drop(columns=['charges'])
y = df['charges']

creamos los conjuntos de entrenamiento y prueba

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

escalamos los datos, esto es necesario para que la red neuronal pueda trabajar con ellos

In [75]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

creamos los tensores de pytorch, estos son los datos que usaremos para entrenar y probar el modelo

In [76]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

creamos la red neuronal, en este caso es una red neuronal simple con 3 capas, la primera capa tiene 32 neuronas, la segunda 16 y la ultima 1, la funcion de activacion es relu para las primeras 2 capas y no hay funcion de activacion para la ultima capa ya que es una regresion.

In [77]:
class SimpleInsuranceNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleInsuranceNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

model = SimpleInsuranceNN(X_train_tensor.shape[1])

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

entrenamos el modelo, en este caso lo entrenamos por 1000 epocas, cada 10 epocas imprimimos el loss para ver como va mejorando el modelo

In [78]:
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.1f}')

Epoch [10/100], Loss: 322382432.0
Epoch [20/100], Loss: 322117152.0
Epoch [30/100], Loss: 321397504.0
Epoch [40/100], Loss: 319802176.0
Epoch [50/100], Loss: 316749376.0
Epoch [60/100], Loss: 311529024.0
Epoch [70/100], Loss: 303333792.0
Epoch [80/100], Loss: 291276544.0
Epoch [90/100], Loss: 274524064.0
Epoch [100/100], Loss: 252680112.0


evaluamos el modelo con los datos de prueba, podemos ver que la perdida es muy alta por lo que el modelo no es muy bueno, hay que recordar que las redes neuronales no son muy buenas en datos tabulares, ademas de esto el dataset es muy pequeño por lo que no hay muchos datos para entrenar el modelo.

In [79]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.1f}')

Test Loss: 249277632.0


para hacer un intento mas con redes neuronales modificaremos los datos para que la variable de salida sea un problema de clasificacion.

para hacer esto lo que haremos es dividir el costo del seguro en 3 categorias, bajo, medio y alto, para esto usaremos los cuartiles de los datos, los datos que esten en el primer cuartil seran bajo (0), los que esten en el segundo cuartil seran medio (1) y los que esten en el tercer cuartil seran alto (2).

In [80]:
Q1 = df['charges'].quantile(0.25)
Q3 = df['charges'].quantile(0.75)

df['category'] = 1
df.loc[df['charges'] < Q1, 'category'] = 0
df.loc[df['charges'] > Q3, 'category'] = 2

hacemos los mismos procesos que antes, solo que ahora la variable de salida es la categoria.

In [81]:
X = df.drop(columns=['charges', 'category'])
y = df['category']

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [84]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).view(-1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).view(-1)

In [85]:
class ClassificationNN(nn.Module):
    def __init__(self, input_dim):
        super(ClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(self.fc3(x), dim=1)

model = ClassificationNN(X_train_tensor.shape[1])

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [86]:
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.9408
Epoch [20/100], Loss: 0.7254
Epoch [30/100], Loss: 0.6653
Epoch [40/100], Loss: 0.6531
Epoch [50/100], Loss: 0.6475
Epoch [60/100], Loss: 0.6454
Epoch [70/100], Loss: 0.6440
Epoch [80/100], Loss: 0.6432
Epoch [90/100], Loss: 0.6426
Epoch [100/100], Loss: 0.6421


In [87]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 89.93%


En esta ultima prueba podemos cer que el modelo tiene una precision buena ahora que solo estamos clasificando si el seguro estara en una de 3 categorias.